**Table of contents**<a id='toc0_'></a>    
- [Automatic differentiation with torch.autograd](#toc1_)    
  - [Tensors, functions, and computational graphs](#toc1_1_)    
  - [Computing gradients](#toc1_2_)    
  - [Disabling gradient tracking](#toc1_3_)    
  - [More on computational graphs](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Automatic differentiation with torch.autograd](#toc0_)

- When training neural networks, the most frequently used algorithm is back propagation.
- 在该算法中，根据损失函数相对于给定参数的梯度来调整参数（模型权重）。
- 损失函数计算神经网络产生的预期输出和实际输出之间的差异。
- 目标是使损失函数的结果尽可能接近于零。
- 该算法通过神经网络向后遍历来调整权重和偏差来重新训练模型。这就是为什么它被称为反向传播。
- 随着时间的推移重新训练模型以将损失减少到 0 的这种前后过程称为梯度下降。

- 为了计算这些梯度，PyTorch 有一个名为 torch.autograd 的内置微分引擎。它支持任何计算图的梯度自动计算。

## <a id='toc1_1_'></a>[Tensors, functions, and computational graphs](#toc0_)

- w and b are parameters, which we need to optimize. 
- Therefore, we need to be able to compute the gradients of loss function with respect to those variables. To do that, we set the requires_grad property of those tensors.


> **Note:** 您可以在创建张量时设置 requires_grad=True 的值，或者稍后使用 x.requires_grad_(True) 方法设置


In [21]:
# 考虑最简单的单层神经网络，具有输入 x 、参数 w 和 b 以及一些损失函数
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output

#  w 和 b 是我们需要优化的参数
# 设置这些张量的 requires_grad 属性
# 也可稍后使用 x.requires_grad_(True) 方法设置
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
loss

tensor(1.4801, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [22]:
print(w)
print(b)
print(z)

tensor([[ 0.6947,  0.8366,  1.8465],
        [ 0.2886,  0.8730, -0.4400],
        [-1.9751,  0.6141, -1.0683],
        [-0.1146,  0.9128,  0.0275],
        [-1.0209, -0.1377,  0.3909]], requires_grad=True)
tensor([-0.5518,  0.0602,  0.0469], requires_grad=True)
tensor([-2.6791,  3.1590,  0.8035], grad_fn=<AddBackward0>)


In [23]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f8a106d3dc0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f8a106d3f40>


## <a id='toc1_2_'></a>[Computing gradients](#toc0_)

- To optimize weights of parameters in the neural network, we need to compute the derivatives of our loss function with respect to parameters, 
- namely, we need $\frac{\partial loss}{\partial w}$ and $\frac{\partial loss}{\partial b}$ under some fixed values of `x` and `y`. 
- To compute those derivatives, we call `loss.backward()`, and then retrieve the values from `w.grad` and `b.grad`:

> 注意：我们只能获取 requires_grad 属性设置为 True 的计算图的叶节点的 grad 属性。此外，出于性能原因，我们只能在给定图上使用 backward 执行一次梯度计算。如果我们需要在同一个图表上执行多个 backward 调用，则需要将 retain_graph=True 传递给 backward 调用。



In [24]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0214, 0.3198, 0.2302],
        [0.0214, 0.3198, 0.2302],
        [0.0214, 0.3198, 0.2302],
        [0.0214, 0.3198, 0.2302],
        [0.0214, 0.3198, 0.2302]])
tensor([0.0214, 0.3198, 0.2302])


In [25]:
print(w)
print(b)
print(z)

tensor([[ 0.6947,  0.8366,  1.8465],
        [ 0.2886,  0.8730, -0.4400],
        [-1.9751,  0.6141, -1.0683],
        [-0.1146,  0.9128,  0.0275],
        [-1.0209, -0.1377,  0.3909]], requires_grad=True)
tensor([-0.5518,  0.0602,  0.0469], requires_grad=True)
tensor([-2.6791,  3.1590,  0.8035], grad_fn=<AddBackward0>)


## <a id='toc1_3_'></a>[Disabling gradient tracking](#toc0_)

- 默认情况下，所有具有 requires_grad=True 的张量都会跟踪其计算历史并支持梯度计算。
- 然而，在某些情况下，我们不需要这样做，例如，当我们训练完模型，只想将其应用于一些输入数据时，即我们只想通过网络进行前向计算。
- 禁用梯度跟踪的原因有
```
> To mark some parameters in your neural network as frozen parameters. This is a very common scenario for fine tuning a pre-trained network
> To speed up computations when you are only doing a forward pass, because computations on tensors that do not track gradients are more efficient.
```


In [11]:
# 1. 可以通过用 torch.no_grad() 块包围我们的计算代码来停止跟踪计算
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

# 2. 获得相同结果的另一种方法是在张量上使用 detach() 方法：
z = torch.matmul(x, w)+b
print(z.requires_grad)
z_det = z.detach()
print(z_det.requires_grad)

True
False
True
False


## <a id='toc1_4_'></a>[More on computational graphs](#toc0_)

In a forward pass, autograd does two things simultaneously:

    runs the requested operation to compute a resulting tensor, and
    maintains the operation’s gradient function in the DAG.

The backward pass kicks off when .backward() is called on the DAG root. autograd then:

    computes the gradients from each .grad_fn,
    accumulates them in the respective tensor’s .grad attribute, and
    using the chain rule, propagates all the way to the leaf tensors.


> **Note** DAGs are dynamic in PyTorch

    An important thing to note is that the graph is recreated from scratch; after each .backward() call, autograd starts populating a new graph. This is exactly what allows you to use control flow statements in your model; you can change the shape, size and operations at every iteration if needed.